# Azure Privileged Identity Management

[Azure AD Privileged Identity Management (PIM)](https://learn.microsoft.com/en-us/entra/id-governance/privileged-identity-management/pim-configure) is a service that enables you to manage, control, and monitor access to important resources in your organization. These resources include resources in Azure AD such as administrative roles and access to Azure resources.

Privileged Identity Management provides time-based and approval-based role activation to mitigate the risks of excessive, unnecessary, or misused access permissions on resources

## This notebook

This notebook will walk you through the following steps:
- Configure the role settings like notifications, role assignment duration, and approval workflow
- Assign eligible roles to users

### Set required variables

This cell sets the variables that are required to run the notebook. The variables are used in the rest of the notebook. 

- The `approversEntraIDGroup` is used in this notebook to identify the Azure Entra ID Group that contains the approvers for the PIM role assignments using the Object Id..
- The `eligiblePrincipalId` is used in this notebook to identify the Azure Entra ID group or user who is able to request a role assignment using the Object Id.
- The `privilegedRoles` denotes roles that require approval before they can become active.


In [ ]:
var tenantId = "";
var subscriptionId = "";
var approversEntraIDGroup = "";
var approversEntraIDGroupName = "";
var eligiblePrincipalId = Guid.Parse("");
var privilegedRoles = new[] { "Owner", "Contributor", "Role Based Access Control Administrator", "User Access Administrator" };

### Set up the ARM Client

The [Microsoft Azure Resource Manager client library for .NET](https://learn.microsoft.com/en-us/dotnet/api/overview/azure/resourcemanager-readme?view=azure-dotnet) allows for inspecting, creating and updating of Azure Resources. The client is used to update PIM role settings and assign roles to groups.

In [ ]:
#r "nuget:Azure.ResourceManager.Authorization"
#r "nuget:Azure.ResourceManager.Resources"
#r "nuget:Azure.Identity"

using Azure;
using Azure.Core;
using Azure.Identity;
using Azure.ResourceManager;
using Azure.ResourceManager.Authorization;

var armClientOptions = new InteractiveBrowserCredentialOptions
{
	TenantId = tenantId,
	AuthorityHost = AzureAuthorityHosts.AzurePublicCloud
};

var armClient = new ArmClient(new InteractiveBrowserCredential(armClientOptions), subscriptionId);
string scope = $"providers/Microsoft.Subscription/subscriptions/{subscriptionId}";
var scopeId = new ResourceIdentifier(string.Format("/{0}", scope));


Build list of roles and their corresponding role policy. Approval is required when the role is in the `privilegedRoles` list.

In [ ]:
var subscription = await armClient.GetDefaultSubscriptionAsync();
var policyApprovalRequired = new Dictionary<string, bool>();

await foreach (var assignments in subscription.GetRoleManagementPolicyAssignments())
{
	var roleName = assignments.Data.PolicyAssignmentProperties.RoleDefinitionDisplayName;
	policyApprovalRequired.Add(assignments.Data.PolicyId.Name, privilegedRoles.Contains(roleName));
}

### Modify default PIM role settings

This code modifies the default [PIM role settings](https://learn.microsoft.com/en-us/entra/id-governance/privileged-identity-management/pim-resource-roles-configure-role-settings#role-settings). The default settings are modified to allow permanent eligible assignments and sets an Azure Entra ID Group with approvers for the role assignments.

In [ ]:
using Azure.ResourceManager.Authorization.Models;

await foreach (var policy in armClient.GetRoleManagementPolicies(scopeId).GetAllAsync())
{
	foreach (var rule in policy.Data.Rules)
	{
		UpdateRule(rule, policyApprovalRequired.GetValueOrDefault(policy.Data.Name));
	}
		
	policy.Update(policy.Data);
}

void UpdateRule(RoleManagementPolicyRule rule, bool isApprovalRequired)
{
	switch (rule)
	{
		case RoleManagementPolicyApprovalRule approvalRule:
			if (approvalRule.Id != "Approval_EndUser_Assignment")
				return;

			approvalRule.Settings.IsApprovalRequired = isApprovalRequired;
						
			var stage = approvalRule.Settings.ApprovalStages.First();
			
			if(!isApprovalRequired) 
				stage.PrimaryApprovers.Clear();
							
			if (stage.PrimaryApprovers.Any(pa => pa.Id == approversEntraIDGroup) || !isApprovalRequired)
				return;

			stage.PrimaryApprovers.Add(new RoleManagementUserInfo
			{
				UserType = RoleManagementUserType.Group,
				Description = approversEntraIDGroupName,
				Id = approversEntraIDGroup,
				IsBackup = false
			});
			break;
		case RoleManagementPolicyNotificationRule notificationRule:
			switch (notificationRule.Id)
			{
				case "Notification_Admin_Admin_Eligibility":
				case "Notification_Admin_Admin_Assignment":
				case "Notification_Admin_EndUser_Assignment":
				case "Notification_Requestor_Admin_Eligibility":
				case "Notification_Requestor_Admin_Assignment":
					notificationRule.AreDefaultRecipientsEnabled = false;
					break;
				default:
					notificationRule.AreDefaultRecipientsEnabled = true;
					break;
			}
			break;
		case RoleManagementPolicyExpirationRule expirationRule:
			expirationRule.IsExpirationRequired = rule.Id != "Expiration_Admin_Eligibility";
			break;
	}
}

### Assing role eligibility to Azure Entra ID Group

Before a user can request a role assignment, the user must be eligible for the role. This code assigns the Azure Entra ID Group to the role eligibility. The group is used to identify the users that are eligible for the role.

In [ ]:
var roleDefinitions = armClient.GetAuthorizationRoleDefinitions(scopeId);
await foreach (var roleDefinition in roleDefinitions.GetAllAsync())
{
    var assignments = armClient.GetRoleEligibilityScheduleRequests(new ResourceIdentifier(scope));
	var eligibilityRequest = new RoleEligibilityScheduleRequestData
	{
		ExpirationType = RoleManagementScheduleExpirationType.NoExpiration,
		RequestType = RoleManagementScheduleRequestType.AdminAssign,
		StartOn = DateTime.Now,
		RoleDefinitionId = roleDefinition.Id,
		PrincipalId = eligiblePrincipalId
	};

	try
	{
		assignments.CreateOrUpdate(WaitUntil.Completed, Guid.NewGuid().ToString(), eligibilityRequest);
	}
	catch(Exception ex)
	{
		Console.WriteLine($"failed to assign role '{roleDefinition.Data.Description}' ({roleDefinition.Data.Name}) to principal {eligiblePrincipalId}: {ex.Message}");
	}
}